In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import asarray
from numpy import save
from numpy import load

import scipy
import seaborn as sns
import itertools
from tqdm import tqdm
from collections import OrderedDict 

import torch
import torch.nn as nn

import pydub
from pydub import AudioSegment

import padertorch as pt
import padercontrib as pc
import paderbox as pb
from padertorch import Model
from paderbox.array import intervall
from padercontrib.database.fearless import Fearless
from padertorch import Model
from paderbox.transform import mfcc
from paderbox.transform import stft,fbank

In [3]:
from padercontrib.database.fearless import Fearless
Fearless = Fearless()
FearlessData = Fearless.data
devList=list(FearlessData['datasets']['Dev_segment'].items())
#devList=list(FearlessData['datasets']['Dev_segment'].items())
devSegLst=[]
for a,b in devList:
    devSegLst.append(b)
dfS = pd.DataFrame(devSegLst)

dfS['vector_path']=np.nan
dfS['intersection_ids']=np.nan
dfS['d_vector']=np.nan

np.nan_to_num(dfS['d_vector'], copy=False)

test_list = torch.tensor(dfS['d_vector'], dtype=torch.float)

intersection_set = load('intersection_set.npy')

for i in range(len(dfS)):
    dfS.loc[i,'intersection_ids'] = dfS['speaker_id'][i] in intersection_set
dfS_filtered_2 = dfS[dfS['intersection_ids'] == True]
dfS_filtered = dfS_filtered_2[dfS_filtered_2['num_samples'] > 4000]
dfS_filtered_RI_sort = dfS_filtered.sort_values(by=['speaker_id'], ignore_index=True)
dfS_filtered_RI_sort.head()

,audio_path,num_samples,speaker_id,transcription,vector_path,intersection_ids,d_vector
0,{'observation': '/net/db/fearless/Audio/Segmen...,70000,AFD1,NO WAIT UNTIL AFTER LAUNCH AND WE GET TRANSMIT...,NaN,True,0.0
1,{'observation': '/net/db/fearless/Audio/Segmen...,18880,AFD1,[unk] A F D ON YOUR LOOP,NaN,True,0.0
2,{'observation': '/net/db/fearless/Audio/Segmen...,10000,AFD1,GO NETWORK,NaN,True,0.0
3,{'observation': '/net/db/fearless/Audio/Segmen...,27040,AFD1,PROCEDURES A F D ON A F D CONFERENCE,NaN,True,0.0
4,{'observation': '/net/db/fearless/Audio/Segmen...,19440,AFD1,MOCR SYSTEMS ONE AND THREE,NaN,True,0.0


In [4]:
a = dfS_filtered_RI_sort['speaker_id'][:10]
b = dfS_filtered_RI_sort['speaker_id'][18:28]
c = dfS_filtered_RI_sort['speaker_id'][29]


In [5]:
from sklearn.metrics.pairwise import cosine_similarity as cs
from sklearn.metrics.pairwise import euclidean_distances as ed

In [1]:
b

NameError: name 'a' is not defined

In [2]:
dvec = np.load('d_vector_train_seg_fbank.npy')

In [77]:
dvec0 = dvec[0:10]
dvec1 = dvec[18:28]

dvec0.shape

(10, 100)

In [83]:
cs1 = cs(dvec0[0:5],dvec0[5:10])
cs2 = cs(dvec0[0:5],dvec1[0:5])
cs1,cs2

(array([[0.98602312, 0.992681  , 0.99603263, 0.99865986, 0.9940386 ],
        [0.99914289, 0.99985858, 0.9991107 , 0.99673304, 0.99971369],
        [0.9989834 , 0.99963813, 0.99916072, 0.99684137, 0.9997727 ],
        [0.99696769, 0.9995068 , 0.99997166, 0.99920254, 0.99975684],
        [0.9962971 , 0.9991532 , 0.99981408, 0.99951051, 0.9993948 ]]),
 array([[0.99783311, 0.99699244, 0.99988115, 0.99941951, 0.99721631],
        [0.99765332, 0.99847894, 0.98977447, 0.99501512, 0.99831665],
        [0.99767741, 0.99852425, 0.99013541, 0.99518675, 0.99840137],
        [0.99961076, 0.99988771, 0.99470019, 0.9982495 , 0.99984529],
        [0.99984115, 0.99996286, 0.99547219, 0.9987497 , 0.99994042]]))

In [ ]:
cs

In [14]:
#167 dimensional d-vector from ang softmax output, 167 dimensional onehot ground-truth labels, roc curve, eer

In [13]:
#100 dimensional d-vector from a layer before the output of ang softmax, 
#similarity between same speaker of train and dev dataset and assign label to dev d-vectors
#compare between predicted assignment and groundtruth label of dev.